# TOFPET charge with LEDs

This notebook is an example of a TOFPET study when using LEDs.

We analyze the integration window and the efine corrected we obtain when a LED or LEDs are switched on depending on the channel position.

We also obtain some plots to show the light distribution in the SiPMs panel.

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pylab as plt
from matplotlib import colors
%matplotlib inline

# Process Data

In [ ]:
files = glob('/analysis/11902/hdf5/proc/linear_interp/files/*h5')
files.sort()
files

In [ ]:
store = pd.HDFStore(files[0], 'r')
store.keys()

In [ ]:
df = pd.read_hdf(files[0],'data_0')
df

**Check tofpet_id, channels_id and integration window size**

In [ ]:
df.tofpet_id.unique()

In [ ]:
df.channel_id.hist(bins = 64)
plt.title('Channels')
plt.xlabel('channel_id')
plt.ylabel('Entries')

In [ ]:
df.intg_w.hist(bins = 100)
plt.title('Integration window for all channels')
plt.xlabel('Integration window')
plt.ylabel('Entries')
plt.yscale('log')

intg_w_unique = df.intg_w.unique()
intg_w_unique.sort()

print('We have selected a fixed integration window of 80 cycles and there are ', 
      len(intg_w_unique), ' different integration windows: \n', '\n', intg_w_unique)

# Studying all channels

**Efine corrected for all channels**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13,4))

fig.suptitle('Efine corrected for all channels')
ax1.hist(df.efine_corrected, bins = 100, label = 'log')
ax1.set_yscale('log')
ax1.set_xlabel('Efine corrected')
ax1.set_ylabel('Entries')
ax1.legend()

ax2.hist(df.efine_corrected, bins = 100, label = 'linear')
ax2.set_xlabel('Efine corrected')
ax2.set_ylabel('Entries')
ax2.legend()

**Efine corrected for all channels selecting the fixed integration window size**

In [ ]:
df[(df['intg_w'] < 85) & (df['intg_w'] > 75)].efine_corrected.hist(bins = 100, alpha = 0.5,label = 'Correct intg_w')
df[(df['intg_w'] > 85)].efine_corrected.hist(bins = 100, alpha = 0.5, label = 'Wrong intg_w')

plt.yscale('log')
plt.xlabel('Efine corrected')
plt.ylabel('Entries')
plt.legend()

# Studying central channels

**Integration window for central channels**

In [ ]:
df[df['channel_id'] == 31].intg_w.hist(bins = 100, label = 'channel 31', alpha = 0.5)
df[df['channel_id'] ==  5].intg_w.hist(bins = 100, label = 'channel  5', alpha = 0.5)
df[df['channel_id'] == 40].intg_w.hist(bins = 100, label = 'channel 40', alpha = 0.5)
df[df['channel_id'] == 35].intg_w.hist(bins = 100, label = 'channel 35', alpha = 0.5)

plt.legend()
plt.title('Integration window for central channels')
plt.xlabel('Integration window')
plt.ylabel('Entries')
plt.yscale('log')

**Efine corrected for central channels**

In [ ]:
df[df['channel_id'] == 31].efine_corrected.hist(bins = 100, label = 'channel 31', alpha = 0.5)
df[df['channel_id'] ==  5].efine_corrected.hist(bins = 100, label = 'channel  5', alpha = 0.5)
df[df['channel_id'] == 40].efine_corrected.hist(bins =  80, label = 'channel 40', alpha = 0.5)
df[df['channel_id'] == 35].efine_corrected.hist(bins =  40, label = 'channel 35', alpha = 0.5)

plt.legend()
plt.title('Efine corrected for central channels')
plt.xlabel('Efine corrected')
plt.ylabel('Entries')
plt.yscale('log')

# Studying corner channels

**Integration window for corner channels**

In [ ]:
df[df['channel_id'] == 10].intg_w.hist(bins = 100, label = 'top-left 10', alpha = 0.5)
df[df['channel_id'] == 24].intg_w.hist(bins = 200, label = 'top-right 24', alpha = 0.5)
df[df['channel_id'] == 51].intg_w.hist(bins = 100, label = 'bottom-left 51', alpha = 0.5)
df[df['channel_id'] == 56].intg_w.hist(bins = 100, label = 'bottom-right 56', alpha = 0.5)

plt.title('Integration window size for corner channels')
plt.xlabel('Integration window')
plt.ylabel('Entries')
plt.yscale('log')
plt.legend()

**Efine corrected for corner channels**

In [ ]:
df[df['channel_id'] == 10].efine_corrected.hist(bins = 200, label = 'top-left 10', alpha = 0.5)
df[df['channel_id'] == 24].efine_corrected.hist(bins = 100, label = 'top-right 24', alpha = 0.5)
df[df['channel_id'] == 51].efine_corrected.hist(bins =  50, label = 'bottom-left 51', alpha = 0.5)
df[df['channel_id'] == 56].efine_corrected.hist(bins = 100, label = 'bottom-right 56', alpha = 0.5)

plt.title('Efine corrected for corner channels')
plt.xlabel('Efine corrected')
plt.ylabel('Entries')
plt.yscale('log')
plt.legend()

# **Efine corrected colour map**

In [ ]:
df_filtered = df[(df['intg_w'] > 75) & (df['intg_w'] < 85)]
print('Number of sensors: ',len(df_filtered.sensor_id.unique()))

In [ ]:
plt.hist(df_filtered[df_filtered['channel_id'] == 34].efine_corrected, bins = 50)
plt.xlabel('Efine corrected')
plt.ylabel('Entries')
plt.title('Efine corrected for channel 34')

In [ ]:
def filtered_mean(df):
    '''In each channel we have got two well differentiated 
    distributions. With this function, we filter the gaussian
    which is due to LED contribution only.'''
    
    mean_vector = 0
    if df.mean() > 30:
        mean_vector = df[df > 30].mean() #we select the gaussian on the right
    elif df.mean() < 30:
        mean_vector = df.mean() # we select all data
        
    return mean_vector
        

We obtain the charge mean for each channel and we create a matrix with the correct position:

In [ ]:
channel_means = df_filtered.groupby('sensor_id')['efine_corrected'].apply(filtered_mean).reset_index()
charge_mean   = np.array(channel_means['efine_corrected']).reshape(8,8)

**Mean charge per channel**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax      = plt.imshow(charge_mean)

cbar    = plt.colorbar()
cbar.set_label('Mean charge (efine corrected)', fontsize = 14)

plt.xlabel('Position sensor x', fontsize = 14)
plt.xticks(np.arange(0, 8, 1))
plt.ylabel('Position sensor y', fontsize = 14)
plt.yticks(np.arange(0, 8, 1))

plt.gca().xaxis.set_label_position('top')
plt.gca().xaxis.tick_top()